# Projeto 1 - Ciência dos Dados

Nome: Beatriz Borges Zackiewicz

Nome: Renato Guedes Passarelli

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download pt_core_news_lg


     -------------------------------------- 568.2/568.2 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [2]:

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\renat\OneDrive\Área de Trabalho\projeto 1 cdados\22-2a-cd-p1-grupo_renatogp\data


Carregando a base de dados com as notícias classificadas manualmente:

In [4]:
filename = 'dados.xlsx'

In [5]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,MT Labs,Pix agendado: você foi selecionado para ter a ...,Atenção: se o algoritmo do Google entregou ess...,13/07/2022 - 15:30,51,0
1,Economia,Gestores ‘de lado’ no câmbio com fim do ciclo ...,Uma corrida eleitoral mais acirrada do que o e...,09/08/2022 - 15:45,19,1
2,Ibovespa Futuro,Ibovespa futuro abre esta sexta-feira em leve ...,O Ibovespa futuro operava em queda na abertura...,08/07/2022 - 9:08,58,1
3,Café,Contratos futuros de café arábica ampliam perd...,Os contratos futuros do café arábica na ICE fe...,02/09/2022 - 17:32,1,0
4,Empresas,"Com prejuízo bilionário, Gol vai cortar voos; ...","Após anunciar um prejuízo de R$ 2,85 bilhões n...",30/07/2022 - 15:33,28,1


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Mercados,"Em dia sem direção definida, Ibovespa (IBOV) f...",O Ibovespa (IBOV) fechou esta terça-feira (12)...,12/07/2022 - 17:08,53,0
1,Commodities,Soja e milho se ajustam cautelosamente esperan...,No aguardo da direção que os mercados financei...,05/08/2022 - 8:08,22,1
2,Economia,Banco Central: Previsão de alta do IPCA em 202...,Os economistas ouvidos pelo Banco Central revi...,18/07/2022 - 8:34,46,0
3,Eleições 2022,Stuhlberger vê risco de ‘república das bananas...,"Sócio da Verde Asset Management, o gestor Luis...",04/08/2022 - 7:19,24,0
4,Mercados,"Dólar reage ao Copom e inflação, mas fica bara...","O dólar comercial sobe 0,5%, perto de R$ 5,14,...",09/08/2022 - 11:43,19,1


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).
Nosso assunto escolhido é sobre o Dólar. Vamos considerar como relevates os seguintes pontos:
-Alta
-Baixa
-Queda
-Crescimento
-Mercado
-Internacional
-Estados Unidos
-Instabilidade
-Moeda
-Valor
-Preço
-Dinheiro

As irrelevantes seriam todas que não se encaixam nos padrões acima


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:

from IPython.display import display
pd.options.display.max_rows = 13
import re 
import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')


cleanwords = ['o','nesta','que','r','após','desta']
for i in cleanwords:
    stopwords.append(i)

punctuation = '[’‘''!-.:?;]'
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[’‘''!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    
print(stopwords)


['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\renat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import spacy
nlp= spacy.load('pt_core_news_lg')
from functools import total_ordering
titulo = train.Titulo.apply(cleanup) # .apply() aplica a função em cada valor da série

texto_e_categorias = train.loc[:,['Titulo','PrimeiroParag','Target']]
#print(texto_e_categorias.head(10))

#primeiro_paragraf = list(train.PrimeiroParag)
#clean_primeiro = []
#print(len(primeiro_paragraf))

tit = list(train.Titulo)
p1o = list(train.PrimeiroParag)
trgt = list(train.Target)


# ideia2 - juntar todas as palavras de cada target e ver as suas frequências
relevante_palavras = []
irrelevante_palavras = [] 
total_vocabulario = []

for i in range(len(train.Titulo)):
    tit_noticiai =  cleanup(str(tit[i]))
    p1o_noticiai =  cleanup(str(p1o[i]))
    tipo = trgt[i]
    if tipo == 1:
        relevante_palavras.append(tit_noticiai.lower().split())
        relevante_palavras.append(p1o_noticiai.lower().split())
    else: 
       irrelevante_palavras.append(p1o_noticiai.lower().split())
       irrelevante_palavras.append(tit_noticiai.lower().split())

    total_vocabulario.append(tit_noticiai.lower().split())
    total_vocabulario.append(p1o_noticiai.lower().split())

def limpa_stopword(lista):
    string = ''
    for frase in lista:
        for palavra in frase:
            if palavra not in stopwords:
                string += palavra
                string += ' '
    return string

total_voc_limpo=limpa_stopword(total_vocabulario)
relevante_limpo=limpa_stopword(relevante_palavras)
irrelevante_limpo=limpa_stopword(irrelevante_palavras)
voc_nlp=nlp(total_voc_limpo)
ire_nlp=nlp(irrelevante_limpo)
rel_nlp=nlp(relevante_limpo)

def spacyy(text):
    empty_list = []
    for token in text:
        empty_list.append(token.lemma_)
    final_string = ' '.join(map(str,empty_list))
    return final_string


voc_final=spacyy(voc_nlp)
ire_final=spacyy(ire_nlp)
rel_final=spacyy(rel_nlp)

total_voc_limpo=pd.Series(voc_final.split())
relevantes_limpo=pd.Series(rel_final.split())
irelevantes_limpo=pd.Series(ire_final.split())

In [9]:
# Frequências:
# Vocabulário total ---- procurar o jeito que tira o de/a/do/e/etc. 
total_voc_limpo.value_counts().head(10)


dólar       174
mercado     147
ibovespa    127
alto         94
dia          87
queda        79
preço        76
futuro       75
subir        69
fechar       62
dtype: int64

In [10]:
# Irrelevantes (Target = 0)
irelevantes_limpo.value_counts().head(10)


mercado     63
ibovespa    49
preço       49
dia         42
alto        41
ano         35
lucro       35
semana      34
ação        34
futuro      33
dtype: int64

In [11]:
# Relevantes (Target = 1)
relevantes_limpo.value_counts().head(10)


dólar       172
mercado      84
ibovespa     78
alto         53
moeda        48
queda        47
dia          45
fechar       43
pregão       43
futuro       42
dtype: int64

In [12]:
# calculo de probabilidades para o suavizador
from ast import Break
from re import X


qtd_relevantes = len(relevantes_limpo) + len(total_voc_limpo) 
qtd_irrelevantes = len(irelevantes_limpo)  + len(total_voc_limpo) 

teste = pd.read_excel(filename, sheet_name = 'Teste')
tit_test = list(test.Titulo)
p1o_test = list(test.PrimeiroParag)
mensagens =[]

i=0
while i < len(tit_test):
    tit_te =  cleanup(str(tit_test[i]))
    p1o_te =  cleanup(str(p1o_test[i]))
    if type(p1o_te) == str: 
        A = tit_te+' '+p1o_te
        mensagens.append(A.lower())
    elif type(p1o_test[i]) == float:
        mensagens.append(tit_te.lower())
    i+=1

final_teste=[]
stringt = ''

for frase in mensagens:
    frase = frase.split() 
    for palavra in frase:
        if palavra not in stopwords:
            stringt += palavra
            stringt += ' '
    text=nlp(stringt)
    empty_list = []
    i=0
    while i < len(text):
        empty_list.append(text[i].lemma_)
        i+=1
    final_string = ' '.join(map(str,empty_list))   
    final_teste.append(final_string)
    stringt = ''
    

classificação = []

i = 0 
while i < len(test.Titulo): 
    P_m_relevante = 1 
    P_m_irrelevante = 1 

    for m in final_teste:
        print(final_teste)
        if m in relevantes_limpo:
            x = relevantes_limpo.value_counts()[m]
            P_m_relevante *= (x+1)/qtd_relevantes 
        elif m not in relevantes_limpo:
            P_m_relevante *= (0 + 1)/qtd_relevantes

        if m in irelevantes_limpo:
            xi = irelevantes_limpo.value_counts()[m]
            P_m_irrelevante *= (xi+1)/qtd_irrelevantes
        elif m not in irelevantes_limpo:
            P_m_irrelevante *= (0+1)/qtd_irrelevantes 

    if P_m_relevante > P_m_irrelevante:
        classificação.append(1)
    elif P_m_irrelevante > P_m_relevante:
        classificação.append(0)
    
    i += 1 

  
print(classificação)

            

       





___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
print(len(final_teste))
print(final_teste[240])
print(len(teste))

240


IndexError: list index out of range

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**